In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font='IPAexGothic')
import japanize_matplotlib
import glob
import pickle

In [41]:
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 75)

In [30]:
#企業コードを書いたexcelファイルを読み込む
companies_df = pd.read_excel('code.xlsx', index_col=0, header=0)
#企業コードをリスト化
code_list = companies_df.index.to_list()
companies_df

,企業名,平均給料(就職四季報),非総合職
銘柄コード,,,
3402,東レ,975,NaN
3405,クラレ,1048,NaN
3407,旭化成,904,NaN
4004,レゾナック,959,NaN
4005,住友化学,982,NaN
4021,日産化学,1010,NaN
4042,東ソー,925,NaN
4063,信越化学工業,886,1.0
4088,エアウォーター,877,NaN


In [31]:
# 読み込むファイル名を指定
filename = 'df_dict.pkl'

# バイナリモードでファイルを開き、オブジェクトを読み込む
with open(filename, 'rb') as file:
    df_dict = pickle.load(file)

In [42]:
df_dict['4088']

,決算年度,決算四半期,売上高,売上原価,売上総利益,売上総利益率,販管費,営業利益,営業利益率,EBITDA,経常利益,法人税等,実質実効税率,親会社に帰属する純利益,純利益率,EPS,BPS,企業価値,時価総額,純有利子負債,PER（会予）,PBR,PER × PBR,EV/EBITDA（会予）,PSR,PCFR,一株当たり配当金,配当利回り（実績）,配当性向,自己資本配当率,ROE,総資産回転率,財務レバレッジ,実質ROE,ROIC,ROA,総資産,流動資産,現金及び預金,現金/総資産,現金/月間売上,現金/時価総額,売掛金および受取手形,製品および仕掛品,流動比率,固定資産,負債,流動負債,買掛金および支払手形,固定負債,有利子負債,D/Eレシオ,ネットD純利益倍率,自己株調整済負債比率,純資産,株主資本,自己資本,自己資本比率,資本金,利益剰余金,営業CF,減価償却費,のれん償却費,投資CF,財務CF,フリーCF,設備投資額,研究開発費,発行済株式総数,従業員数,売上/従業員数,営利/従業員数,株価,売上増加率
0,2023,4.0,1.024540e+12,8.042710e+11,2.202680e+11,21.5,1.584110e+11,6.827200e+10,6.70,1.148620e+11,6.671200e+10,2.056500e+10,30.80,4.436000e+10,4.30,194.40,2140.70,9.003446e+11,5.466006e+11,3.419940e+11,10.90,1.10,12.20,7.20,0.50,5.70,64.0,2.670,32.9,3.00,9.10,0.84,2.50,9.00,5.3,3.60,1.222696e+12,4.493720e+11,6.497500e+10,5.0,1.0,12.0,2.405840e+11,9.264300e+10,141.50,7.733230e+11,7.142100e+11,3.174810e+11,1.676850e+11,3.967290e+11,4.187190e+11,0.9,7.70,140.0,5.084850e+11,4.885570e+11,4.885580e+11,40.0,5.585500e+10,3.351130e+11,7.962500e+10,4.659000e+10,NaN,-9.796600e+10,1.472300e+10,1.315700e+10,6.646800e+10,6.172000e+09,228225740.0,20348.0,50350894.0,3355219.0,2395.0,NaN
1,2022,4.0,1.004914e+12,8.048300e+11,2.000840e+11,19.9,1.503790e+11,6.218100e+10,6.20,1.071680e+11,6.097800e+10,1.802300e+10,29.60,4.013700e+10,4.00,176.50,1892.40,6.781861e+11,3.774051e+11,2.946300e+11,8.60,0.90,7.50,5.80,0.30,4.20,60.0,3.610,34.0,3.20,9.30,0.92,2.50,9.30,5.5,3.70,1.091645e+12,4.311390e+11,6.594400e+10,6.0,1.0,17.0,2.292760e+11,9.201400e+10,143.90,6.605050e+11,6.451620e+11,2.996970e+11,1.666010e+11,3.454650e+11,3.667250e+11,0.9,7.30,143.0,4.464820e+11,4.302320e+11,4.302330e+11,39.0,5.585500e+10,3.036800e+11,5.695300e+10,4.498700e+10,NaN,-7.113500e+10,1.925700e+10,-9.716000e+09,6.666900e+10,5.451000e+09,227352444.0,20109.0,49973345.0,3092198.0,1660.0,1.953003
2,2021,4.0,8.886680e+11,6.895550e+11,1.991120e+11,22.4,1.397030e+11,6.517400e+10,7.30,1.085520e+11,6.423000e+10,1.782300e+10,27.70,4.321400e+10,4.90,190.80,1744.40,6.765465e+11,3.896005e+11,2.816230e+11,9.70,1.00,9.60,6.40,0.40,4.70,56.0,3.260,29.4,3.20,10.90,0.87,2.60,10.80,6.4,4.20,1.022031e+12,3.768890e+11,5.955400e+10,6.0,1.0,15.0,2.030490e+11,7.640400e+10,143.30,6.451410e+11,6.021740e+11,2.630450e+11,1.490980e+11,3.391290e+11,3.465000e+11,0.9,6.50,142.0,4.198570e+11,3.951310e+11,3.951320e+11,39.0,5.585500e+10,2.751580e+11,7.157200e+10,4.337800e+10,NaN,-5.315400e+10,-6.622000e+09,2.523700e+10,4.633500e+10,5.338000e+09,226511894.0,19560.0,45432924.0,3332004.0,1720.0,13.080926
3,2020,4.0,8.066300e+11,6.257340e+11,1.808950e+11,22.4,1.353980e+11,5.123100e+10,6.40,9.026400e+10,4.965100e+10,1.929200e+10,38.90,2.736700e+10,3.40,121.20,1584.90,6.995030e+11,4.379740e+11,2.569390e+11,10.20,1.20,12.50,6.70,0.50,5.30,44.0,2.270,36.3,2.80,7.60,0.87,2.60,7.50,4.8,3.00,9.268210e+11,3.357670e+11,4.598300e+10,5.0,1.0,10.0,1.886640e+11,6.767900e+10,140.40,5.910530e+11,5.544310e+11,2.391280e+11,1.357160e+11,3.153020e+11,3.075120e+11,0.9,9.40,147.0,3.723890e+11,3.577970e+11,3.577980e+11,39.0,5.585500e+10,2.447940e+11,7.660100e+10,3.903300e+10,NaN,-5.269900e+10,-2.088900e+10,2.457400e+10,5.202700e+10,3.978000e+09,225759798.0,18843.0,42807939.0,2718835.0,1940.0,10.170462
4,2019,4.0,8.090830e+11,6.284630e+11,1.806200e+11,22.3,1.353830e+11,5.061600e+10,6.30,8.561000e+10,4.983000e+10,1.608500e+10,32.30,3.043000e+10,3.80,133.80,1460.00,5.970779e+11,3.379049e+11,2.533790e+11,11.70,1.00,11.90,7.00,0.40,5.30,44.0,2.960,32.9,3.00,9.20,0.90,2.70,9.10,5.5,3.40,8.996990e+11,3.332770e+11,4.186100e+10,5.0,1.0,12.0,1.874020e+11,6.441500e+10,117.60,5.664220e+11,5.478840e+11,2.835000e+11,1.379450e+11,2.643830e+11,3.010340e+11,0.9,8.30,155.0,3.518150e+11,3.319920e+11,3.319930e+11,37.0,5.585500e+10,2.288540e+11,4.378400e+10,3.499400e+10,NaN,-1.155970e+11,8.098100e+10,-2.040300e+10,6.418700e+10,3.422000e+09,227392253.0,18211.0,44428258.0,2779419.0,1486.0,-0.303183
5,5年平均値,4.0,9.067670e+11,7.105706e+11,1.961958e+11,21.7,1.438548e+11,5.949480e+10,6.58,1.012912e+11,5.

In [32]:
#PERトレンドグラフ(5年分)を作成
years = [2019, 2020, 2021, 2022, 2023]

# code_list 内の各企業コードに対して処理を実行
for target in code_list:
    # companies_df に企業コードが存在しなければスキップ
    if str(target) not in df_dict.keys():
        continue
    
    # グラフの作成開始
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # companies_df から企業名を取得（インデックスが整数であることを前提）
    company_name = companies_df.loc[int(target), '企業名']

    # 第1の y 軸を設定：売上総利益率のプロット
    color1 = 'tab:blue'
    ax1.set_xlabel('年度', fontsize=14)
    ax1.set_ylabel('売上総利益率（粗利率）[%]', color=color1, fontsize=14)
    ax1.plot(df_dict[str(target)]['決算年度'][:-1],
             df_dict[str(target)]['売上総利益率'][:-1],
             color=color1, marker='o', label='売上総利益率')
    ax1.tick_params(axis='y', labelcolor=color1, labelsize=12)

    # x 軸の目盛りを指定の年に設定
    ax1.set_xticks(years)
    ax1.set_xticklabels([str(year) for year in years], fontsize=12)

    # 第2の y 軸を設定：PER（会予）のプロット
    ax2 = ax1.twinx()  # 共有 x 軸の第2 y 軸を作成
    color2 = 'tab:red'
    ax2.set_ylabel('PER（会予）', color=color2, fontsize=14)
    ax2.plot(df_dict[str(target)]['決算年度'][:-1],
             df_dict[str(target)]['PER（会予）'][:-1],
             color=color2, marker='s', label='PER（会予）')
    ax2.tick_params(axis='y', labelcolor=color2, labelsize=12)

    # グラフタイトルを設定
    plt.title(f'{str(target)} {company_name}', fontsize=16)
    plt.grid(False)
    fig.tight_layout()

    # グラフをPNGファイルとして保存
    plt.savefig(f'./graph/trend/5year_trend_{str(target)}.png', dpi=150, facecolor='white')
    plt.close()  # 次のループのために現在の図を閉じる

In [33]:
index_to_label = {
    0: 'result_df_2023',
    1: 'result_df_2022',
    2: 'result_df_2021',
    3: 'result_df_2020',
    4: 'result_df_2019',
    5: 'result_df_5mean'
}

In [34]:
# 各結果データフレームを格納するための辞書
results = {}

# 各インデックスに対する処理を行う
for idx, df_name in index_to_label.items():
    records = []

    for code in code_list:
        # companies_df に企業コードが存在しなければスキップ
        if str(code) not in df_dict.keys():
            continue
        # 現在の企業コードに対応するデータフレームの idx 行目を取得
        row = df_dict[str(code)].iloc[idx]

        # 必要な値を抽出
        gross_profit_rate = row['売上総利益率']
        per_value = row['PER（会予）']

        # 辞書形式で記録を保存
        records.append({
            'code': code,
            '売上総利益率': gross_profit_rate,
            'PER（会予）': per_value
        })

    # リストからデータフレームを作成し、'code' をインデックスに設定
    df_temp = pd.DataFrame(records).set_index('code')

    # インデックスを企業名に変換
    df_temp.index = df_temp.index.map(companies_df['企業名'])

    # 'PER（会予）' 列を数値型に変換
    df_temp['PER（会予）'] = pd.to_numeric(df_temp['PER（会予）'], errors='coerce')

    # 作成したデータフレームを辞書に保存
    results[df_name] = df_temp

In [35]:
for df in results.values():   
    # NaN を含む行をすべて削除
    df.dropna(inplace=True)
    
    # 'PER（会予）' が 70 以上の行を削除
    if 'PER（会予）' in df.columns:
        condition = df['PER（会予）'] >= 70
        df.drop(df[condition].index, inplace=True)

In [36]:
# results 辞書内の各データフレームについて処理
for key, df in results.items():
    # 新しい図を作成
    plt.figure(figsize=(10, 8))

    # 散布図を作成：x軸に 'PER（会予）', y軸に '売上総利益率'
    plt.scatter(df['PER（会予）'], df['売上総利益率'], s=100)

    # 各点に企業名を注釈として追加
    for company, row in df.iterrows():
        # 数値が NaN でないことを確認してから注釈を追加
        if pd.notna(row['PER（会予）']) and pd.notna(row['売上総利益率']):
            plt.annotate(company, 
                         (row['PER（会予）'], row['売上総利益率']),
                         textcoords="offset points",  
                         xytext=(5,5),               
                         ha='left',                  
                         fontsize=14)

    # 軸ラベルを設定
    plt.xlabel('PER（会予）', fontsize=16)
    plt.ylabel('売上総利益率（粗利率）[%]', fontsize=16)

    # メモリラベルのフォントサイズを設定
    plt.tick_params(axis='both', which='major', labelsize=14)

    # グリッドを表示
    plt.grid(True)

    # 余白を自動調整
    plt.tight_layout()

    # キーに基づいて注釈テキストを設定（例：result_df_2023 → "2023年度のデータです"）
    # キーから年度情報を抽出
    if '5mean' in key:
        note = '注: データは5年平均です'
    else:
        # 例: 'result_df_2023' から '2023年度のデータです' を生成
        year = key.replace('result_df_', '')
        note = f'注: {year}年度のデータです'

    # グラフ下部に注釈を追加
    plt.figtext(0.8, 0.01, note, wrap=True, 
                horizontalalignment='center', fontsize=12, color='gray')

    # ファイル名をキーに基づいて決定し、PNG ファイルとして保存
    filename = f'./graph/PER/arari_PER_{key}.png'
    plt.savefig(filename, dpi=150, facecolor='white')
    plt.close()  # 次の図のために現在の図を閉じる

In [37]:
results['result_df_2023'].corr()

,売上総利益率,PER（会予）
売上総利益率,1.00000,0.29621
PER（会予）,0.29621,1.00000


In [38]:
results['result_df_5mean'].corr()

,売上総利益率,PER（会予）
売上総利益率,1.000000,0.304426
PER（会予）,0.304426,1.000000


In [47]:
# 表示したい年度（例として5年間）
years = [2019, 2020, 2021, 2022, 2023]

# code_list 内の各企業コードに対して処理を実行
for target in code_list:
    # companies_df に企業コードが存在しなければスキップ
    if str(target) not in df_dict.keys():
        continue

    # グラフ作成のための図と軸を生成
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # 企業名の取得（companies_df のインデックスが整数である前提）
    company_name = companies_df.loc[int(target), '企業名']

    # 企業の各指標のデータを取得（末尾の1行が不要な場合はスライスで除外）
    # ※ここでは決算年度のリストを使ってx軸を設定します。
    #fiscal_years = df_dict[str(target)]['決算年度'][:-1]
    #revenue = df_dict[str(target)]['売上高'][:-1]
    #gross_profit = df_dict[str(target)]['売上総利益'][:-1]
    #gross_profit_margin = df_dict[str(target)]['売上総利益率'][:-1]
    fiscal_years = list(reversed(df_dict[str(target)]['決算年度'][:-1]))
    revenue = list(reversed(df_dict[str(target)]['売上高'][:-1]))
    gross_profit = list(reversed(df_dict[str(target)]['売上総利益'][:-1]))
    gross_profit_margin = list(reversed(df_dict[str(target)]['売上総利益率'][:-1]))

    # x軸の位置を生成（棒グラフを横並びに表示するための位置調整）
    x = np.arange(len(fiscal_years))
    bar_width = 0.35

    # --- ax1: 売上高と売上総利益の棒グラフ ---
    color1 = 'tab:blue'
    ax1.set_xlabel('年度', fontsize=14)
    ax1.set_ylabel('金額', color=color1, fontsize=14)

    # 売上高の棒グラフ（x位置を左にずらす）
    bars1 = ax1.bar(x - bar_width/2, revenue, width=bar_width, color='skyblue', label='売上高')
    # 売上総利益の棒グラフ（x位置を右にずらす）
    bars2 = ax1.bar(x + bar_width/2, gross_profit, width=bar_width, color='dodgerblue', label='売上総利益')

    ax1.tick_params(axis='y', labelcolor=color1, labelsize=12)
    ax1.set_xticks(x)
    # x軸のラベルは fiscal_years を利用（もしくはyearsリストに合わせる場合は調整してください）
    ax1.set_xticklabels(fiscal_years, fontsize=12)

    # --- ax2: 売上総利益率の折れ線グラフ ---
    ax2 = ax1.twinx()  # ax1とx軸を共有する第2のy軸を作成
    color2 = 'tab:red'
    ax2.set_ylabel('売上総利益率（%）', color=color2, fontsize=14)
    line, = ax2.plot(x, gross_profit_margin, color=color2, marker='o', linestyle='-', linewidth=2, label='売上総利益率')
    ax2.tick_params(axis='y', labelcolor=color2, labelsize=12)
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    # グラフタイトルの設定
    plt.title(f'{str(target)} {company_name}', fontsize=16)
    plt.grid(False)
    fig.tight_layout()

    # グラフをPNGファイルとして保存
    plt.savefig(f'./graph/trend/5year_barplot_{str(target)}.png', dpi=150, facecolor='white')
    plt.close()  # 次のループのために現在の図を閉じる